<a href="https://colab.research.google.com/github/Shashwat26Bhatnagar/BitTanh_Action_Function/blob/main/GoingToWharton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install swig

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,028 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/dogs-vs-cats.zip'

In [ ]:
%%writefile approx.c
/* File : approx.c */

    #include <math.h>
    #include <stdio.h>

float custom_tanh(float number) {
    float x_square = pow(number, 2);
    // printf("%f\n",x_square);
    float x_double = 2 * number;
    float y;

    float pos = x_square + x_double;
    float neg = pos + 2;

    long i, j, k;
    i = *(long *) &pos;
    j = *(long *) &neg;

    // Take 2's complement of j
    // i = ~i + 1;

    // Perform bitwise addition
    k = i-j;

    // Add the bias to the result
    k = k + 0x3f811a7a;


    // Convert back to float
    y = *(float *) &k;

    return y;
}




Writing approx.c


In [ ]:
%%writefile approx.i
/* approx.i */
%module approx
%{
    extern float custom_tanh(float number);


%}


extern float custom_tanh(float number);



In [ ]:
!python3-config --cflags

In [ ]:
import sysconfig

print(sysconfig.get_paths()['include'])



In [ ]:
!swig -python  approx.i
!gcc -c approx.c approx_wrap.c \
  -I/usr/include/python3.10 -fPIC
!ld -shared approx.o approx_wrap.o -o _approx.so


!swig -python approx.i
!gcc -c -fPIC approx.c approx_wrap.c -I/usr/include/python3.10
!ld -shared approx.o approx_wrap.o -o _approx.so


In [ ]:
import approx
import numpy as np
import time
from multiprocessing import Process , Array

In [ ]:
def process_element(arr, result, index):
  result[index]=approx.custom_tanh(arr[index])
def parallel_compute(arr):
  result = Array('d',len(arr))#creating a shared libarry for processes
  #creating start processes for each element
  processes=[]
  for i in range(len(arr)):
    process=Process(target=process_element , args=(arr,result,i))
    process.start()
    processes.append(process)

  #wait for all process to complete
  for process in processes:
    process.join()

  #convert the shared array to Numpy array and return it
  return np.frombuffer(result.get_obj())

def BitTanh(arr):
  result_arr= parallel_compute(arr)
  return result_arr


In [ ]:
data= np.array([00.53,0.032,0.02236,0.28868])

In [ ]:
%%timeit
BitTanh(data)

In [ ]:
%%timeit
np.tanh(data)

In [ ]:
import numpy as np
import approx
from multiprocessing import Pool

def custom_activation(x):
    def custom_activation_function(x):
        # Example activation function using math.tanh()
        return approx.custom_tanh(x)

    # for i,xi in x.flatten():
    #   print(type(xi))
    #   xi=xi.detach().numpy()
    #   result_flattened_image=custom_activation_function(xi)
    # Iterate through each element of the input tensor x and apply the custom activation function
    result_flattened_image=[]
    for xi in x.flatten():
      xa=float(xi)
      result_flattened_image.append(approx.custom_tanh(xa))

    #result_flattened_image = [{xa:=float(xi),custom_activation_function(xa)} for xi in x.flatten()]


    #result_image = np.array(result_flattened_image).reshape(x.shape)
    #print(result_flattened_image)
    # result_image = list(map(int, result_flattened_image))
    # result_image = result_flattened_image.astype(np.float32)
    return result_flattened_image
# eg = torch.tensor([1.,2.], requires_grad = True)
# custom_activation(eg)


In [ ]:
# import numpy as np
# from multiprocessing import Pool

# # Define your custom activation function
# def custom_activation(x):
#     # Example custom activation function
#     return x * 2  # Double the input value

# # Function to apply activation function to a chunk of image array
# def apply_activation_to_chunk(chunk):
#     return custom_activation(chunk)

# # Example 3D NumPy array representing an image
# image = np.random.rand(100, 100, 1)  # Example image with dimensions (height, width, channels)

# # Define the chunk size (adjust as needed)
# chunk_size = 10

# # Split the image array into smaller chunks along the height dimension
# chunks = [image[i:i+chunk_size] for i in range(0, image.shape[0], chunk_size)]

# # Create a multiprocessing pool
# with Pool() as pool:
#     # Apply the activation function to each chunk in parallel
#     processed_chunks = pool.map(apply_activation_to_chunk, chunks)

# # Combine the processed chunks to reconstruct the processed image array
# processed_image = np.concatenate(processed_chunks)

# # Processed image is now ready for further use

In [ ]:
!pip install dill

In [ ]:
import os
import torch
import csv
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import dill as pickle

# Define your custom activation function
class CustomActivation(nn.Module):
    def __init__(self):
        super(CustomActivation, self).__init__()

    def forward(self, x):

        a = custom_activation(x)
        return a  # Example of a custom activation function

# Define your custom model
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(784, 512)  # Input size: 28x28 = 784
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)   # Add an extra layer with 64 units
        self.fc5 = nn.Linear(64, 2)     # Output size: 2 for 2 classes (cat and dog)
        self.custom_activation = CustomActivation()  # Custom activation function

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input images
        x = self.custom_activation(self.fc1(x))  # Applying custom activation to fc1
        x=torch.as_tensor(x,dtype=torch.float32)


        x = x.view(-1, 512)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)  # Apply log softmax for classification


# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.transform = transform
        self.image_paths = self.get_image_paths()

    def get_image_paths(self):
        image_paths = []
        for root, _, files in os.walk(self.data_folder):
            for file in files:
                if file.endswith('.jpg') or file.endswith('.png'):
                    image_paths.append(os.path.join(root, file))
        return image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('L')  # Convert image to grayscale
        if self.transform:
           image = self.transform(image)
        label = 0 if 'cat' in os.path.dirname(image_path) else 1  # Inferred label based on directory structure
        return image, label


# Load and preprocess the data
def load_data(data_folder, batch_size=64, test_size=0.2, random_state=42):
    transform = transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    dataset = CustomDataset(data_folder, transform=transform)

    # Split the dataset into train and test sets
    train_set, test_set = train_test_split(dataset, test_size=test_size, random_state=random_state)

    # Create DataLoader objects
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# Train the model
def train_model(model, train_loader, optimizer, criterion, epochs):
    model.train()
    losses = []
    for epoch in range(epochs):
        running_loss = 0.0
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)
        print('Epoch %d, Loss: %.4f' % (epoch+1, epoch_loss))
    return losses

# Test the model
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    print('Accuracy on test set: %d %%' % accuracy)
    return accuracy

# Load data
train_loader, test_loader = load_data('/content/train')

# Define and train the model
model = CustomModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
epochs = 25
losses = train_model(model, train_loader, optimizer, criterion, epochs)
accuracy = test_model(model, test_loader)

# Plot loss vs epochs
plt.plot(range(1, epochs+1), losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.grid(True)
plt.show()

# Plot accuracy vs epochs
plt.plot(range(1, epochs+1), [accuracy]*epochs)
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy vs Epochs')
plt.grid(True)
plt.show()

# Save the current figure
plt.savefig('output_window.png')


In [ ]:
# Plot loss vs epochs
plt.plot(range(1, epochs+1), losses, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.grid(True)
plt.show()

# Plot accuracy vs epochs
plt.plot(range(1, epochs+1), [accuracy]*epochs, linestyle='--', label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy vs Epochs')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Save the model
torch.save(model.state_dict(), 'custom_model.pth')

In [ ]:
def predict_image(model, image_path):
    # Open the image
    image = Image.open(image_path).convert('L')  # Convert image to grayscale
    # Preprocess the image
    transform = transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    # Predict the label
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)

    # Display the image and prediction
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    if predicted.item() == 0:
        print("Predicted: Cat")
    else:
        print("Predicted: Dog")
    plt.show()

# Example usage
image_path = 'example_img3.png'
predict_image(model, image_path)


In [ ]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000

In [ ]:
test_model(model, test_loader)